In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

In [2]:
raw_data = pd.read_csv("IPL_Ball_by_Ball_2008_2022.csv")
raw_data = raw_data.sample(frac=0.025, random_state=42)
raw_data

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
177814,501226,1,12,2,MS Dhoni,Yuvraj Singh,MEK Hussey,NaN,1,0,1,0,0,NaN,NaN,NaN,Chennai Super Kings
3287,1304107,1,17,4,SW Billings,T Natarajan,AD Russell,NaN,6,0,6,0,0,NaN,NaN,NaN,Kolkata Knight Riders
188612,419148,2,13,3,RV Uthappa,AD Mathews,LRPL Taylor,NaN,0,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
96212,980961,1,13,6,RA Jadeja,Z Khan,KD Karthik,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Lions
30637,1254064,1,5,1,RR Pant,JD Unadkat,AM Rahane,NaN,4,0,4,0,0,NaN,NaN,NaN,Delhi Capitals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95356,980969,2,14,1,V Kohli,R Bhatia,SR Watson,NaN,4,0,4,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
216575,336022,1,8,2,V Sehwag,RR Powar,S Dhawan,wides,0,1,1,0,0,NaN,NaN,NaN,Delhi Daredevils
16561,1304052,1,18,5,UT Yadav,Akash Deep,CV Varun,NaN,0,0,0,0,1,UT Yadav,bowled,NaN,Kolkata Knight Riders
8938,1304084,2,13,4,AT Rayudu,Sandeep Sharma,RA Jadeja,NaN,4,0,4,0,0,NaN,NaN,NaN,Chennai Super Kings


In [3]:
def rectify_data(raw_data):
    raw_data=raw_data[raw_data['total_run']<=6]
    processed_data = pd.DataFrame()
    processed_data['first_innings'] = 2-raw_data['innings']
    processed_data['second_innings'] = raw_data['innings']-1
    processed_data['powerplay'] = (raw_data['overs']<=6).astype(int)
    processed_data['middle'] = ((raw_data['overs']>6) & (raw_data['overs']<=15)).astype(int)
    processed_data['death'] = (raw_data['overs']>15).astype(int)
    distinct_batters = raw_data['batter'].unique()
    batter_names = pd.concat([raw_data['batter'].fillna('') == batter for batter in distinct_batters], axis=1).astype(int)
    batter_names.columns = distinct_batters
    processed_data = pd.concat([processed_data,batter_names],axis=1)
    distinct_bowlers = raw_data['bowler'].unique()
    bowler_names = pd.concat([raw_data['bowler'].fillna('') == bowler for bowler in distinct_bowlers], axis=1).astype(int)
    bowler_names.columns = distinct_bowlers
    processed_data = pd.concat([processed_data,batter_names],axis=1)
    # processed_data['run_and_wicket'] = raw_data[['total_run','isWicketDelivery']].apply(list,axis=1)
    processed_data['label'] = raw_data['total_run']+7*raw_data['isWicketDelivery']

    #outliers
    # processed_data = processed_data[processed_data['label']%7<=6]
    
    return processed_data

In [4]:
# def rectify_data(raw_data):
#     raw_data=raw_data[raw_data['total_run']<=6]
#     processed_data = pd.DataFrame()
#     processed_data['first_innings'] = 2-raw_data['innings']
#     processed_data['second_innings'] = raw_data['innings']-1
#     processed_data['powerplay'] = (raw_data['overs']<=6).astype(int)
#     processed_data['middle'] = ((raw_data['overs']>6) & (raw_data['overs']<=15)).astype(int)
#     processed_data['death'] = (raw_data['overs']>15).astype(int)
#     distinct_batters = {}
#     counter = 0
#     for i in raw_data['batter'].unique() :
#         distinct_batters[i]=counter
#         counter+=1
#     processed_data['batter'] = [distinct_batters[x] for x in raw_data['batter']]
#     distinct_bowlers = {}
#     counter = 0
#     for i in raw_data['bowler'].unique() :
#         distinct_bowlers[i]=counter
#         counter+=1
#     processed_data['bowler'] = [distinct_bowlers[x] for x in raw_data['bowler']]
#     # processed_data['total_run'] = raw_data['total_run']
#     # processed_data['iswicket'] = raw_data['isWicketDelivery']
#     processed_data['label'] = raw_data['total_run']+7*raw_data['isWicketDelivery']
#     return processed_data

In [5]:
modified_data = rectify_data(raw_data)
X = modified_data.drop(columns = "label")
Y = modified_data["label"]
# if pca is required :-
# pca = PCA(n_components=100) 
# X = pca.fit_transform(X)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 1092003)
modified_data

,first_innings,second_innings,powerplay,middle,death,MS Dhoni,SW Billings,RV Uthappa,RA Jadeja,RR Pant,...,RD Chahar,MM Sharma,Virat Singh,M Theekshana,NK Patel,M Prasidh Krishna,AS Joseph,J Suchith,DAJ Bracewell,label
177814,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3287,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6
188612,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
96212,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
30637,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95356,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
216575,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16561,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
8938,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame containing the features and labels

# Separate features (X) and labels (y)
X = modified_data.drop(columns=['label'])  # Assuming 'label' is the column with the target labels
y = modified_data['label']

# Convert target labels to one-hot encoding
Y_one_hot = pd.get_dummies(y)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_one_hot, test_size=0.2, random_state=42)

def sigmoid(z):
    """Sigmoid activation function"""
    return 1 / (1 + np.exp(-z))

def softmax(X):
    """Softmax activation function"""
    exp_X = np.exp(X - np.max(X, axis=1, keepdims=True))
    return exp_X / np.sum(exp_X, axis=1, keepdims=True)

def initialize_parameters(num_features, num_classes):
    """Initialize weights and bias"""
    W = np.random.randn(num_features, num_classes)
    b = np.zeros((1, num_classes))
    return W, b

def forward_propagation(X, W, b):
    """Forward propagation"""
    Z = np.dot(X, W) + b
    A = softmax(Z)
    return A

def custom_logistic_loss(Y, A, label_coefficients):
    """Compute cross-entropy loss with label-specific coefficients"""
    epsilon = 1e-15  # Small constant to prevent log(0)
    A = np.clip(A, epsilon, 1 - epsilon)
    coefficients_array = np.array([label_coefficients[label] for label in range(len(label_coefficients))])
    # Apply label-specific coefficients to the loss
    loss = -np.sum(Y * np.log(A) * coefficients_array) / Y.shape[0]

    return loss

def backward_propagation(X, A, Y):
    """Backward propagation"""
    dZ = A - Y * np.array([label_coefficients[label] for label in range(len(label_coefficients))])
    dW = np.dot(X.T, dZ)
    db = np.sum(dZ, axis=0, keepdims=True)
    return dW, db

def update_parameters(W, b, dW, db, learning_rate):
    """Update parameters using gradient descent"""
    W -= learning_rate * dW
    b -= learning_rate * db
    return W, b

def train(X, Y, num_epochs, learning_rate, label_coefficients):
    num_features = X.shape[1]
    num_classes = Y.shape[1]

    W, b = initialize_parameters(num_features, num_classes)

    for epoch in range(num_epochs):
        # Forward propagation
        A = forward_propagation(X, W, b)

        # Compute loss
        loss = custom_logistic_loss(Y, A, label_coefficients)

        # Backward propagation
        dW, db = backward_propagation(X, A, Y)

        # Update parameters
        W, b = update_parameters(W, b, dW, db, learning_rate)

        # Print loss every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")

    return W, b

def predict(X, W, b):
    """Make predictions and convert to label form"""
    A = forward_propagation(X, W, b)
    predicted_labels = np.argmax(A, axis=1)
    return predicted_labels

# Example usage:
num_epochs = 1000
learning_rate = 0.01

# Assuming label_coefficients is a dictionary specifying coefficients for each label
label_coefficients = {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 20.0, 8: 20.0}

W_trained, b_trained = train(X_train, Y_train.values, num_epochs, learning_rate, label_coefficients)

# Assuming X_test is your test feature matrix
# X_test.shape = (m_test, num_features)

# Make predictions
predicted_labels = predict(X_test, W_trained, b_trained)
print("Predicted Labels:", predicted_labels)


Epoch 0, Loss: 6.606379632243106
Epoch 100, Loss: 46.79286574149917
Epoch 200, Loss: 47.0338293335502
Epoch 300, Loss: 15.459298886500042
Epoch 400, Loss: 27.338980744654418
Epoch 500, Loss: 29.063035723474478
Epoch 600, Loss: 47.384899379225104
Epoch 700, Loss: 20.84344790092674
Epoch 800, Loss: 40.330488040141994
Epoch 900, Loss: 8.29831894836871
Predicted Labels: [6 4 4 ... 4 7 7]


In [7]:
accuracy_score(predicted_labels, Y_test.values.argmax(axis=1))

0.08141592920353982

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 1092003)

In [9]:
def Logistic_Regression(X_train,Y_train):
    model = LogisticRegression(n_jobs=-1,max_iter=100000)
    model.fit(X_train,Y_train)
    return model

In [10]:
def SVM(X_train,Y_train):
    model = SVC(probability=True)
    model.fit(X_train,Y_train)
    return model

In [11]:
def MLP(X_train,Y_train):
    model = MLPClassifier(activation = 'relu',hidden_layer_sizes=(10,5,10), max_iter=30000)
    model.fit(X_train,Y_train)
    return model

In [12]:
print("Using Logistic Regression :-")
model_lr = Logistic_Regression(X_train,Y_train)
Y_train_pred = model_lr.predict(X_train)
Y_test_pred = model_lr.predict(X_test)
print("Accuracy on Trained data - ",accuracy_score(Y_train,Y_train_pred))
print("Classification report on Trained data - \n",classification_report(Y_train,Y_train_pred))
print("Confusion matrix on Trained data - \n",confusion_matrix(Y_train,Y_train_pred))
print("Accuracy on Test data - ",accuracy_score(Y_test,Y_test_pred))
print("Classification report on Test data - \n",classification_report(Y_test,Y_test_pred))
print("Confusion matrix on Test data - \n",confusion_matrix(Y_test,Y_test_pred))

Using Logistic Regression :-
Accuracy on Trained data -  0.4903739765434831
Classification report on Trained data - 
               precision    recall  f1-score   support

           0       0.48      0.50      0.49      1374
           1       0.49      0.79      0.60      1865
           2       0.48      0.04      0.07       278
           3       0.00      0.00      0.00        14
           4       0.74      0.06      0.12       538
           5       0.00      0.00      0.00         7
           6       1.00      0.01      0.03       207
           7       0.92      0.05      0.09       225
           8       0.00      0.00      0.00        11

    accuracy                           0.49      4519
   macro avg       0.46      0.16      0.16      4519
weighted avg       0.56      0.49      0.42      4519

Confusion matrix on Trained data - 
 [[ 689  677    3    0    5    0    0    0    0]
 [ 390 1469    3    0    3    0    0    0    0]
 [  74  194   10    0    0    0    0    0   

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metr

In [13]:
print("Using SVM :-")
model_SVM = SVM(X_train,Y_train)
Y_train_pred = model_SVM.predict(X_train)
Y_test_pred = model_SVM.predict(X_test)
print("Accuracy on Trained data - ",accuracy_score(Y_train,Y_train_pred))
print("Classification report on Trained data - \n",classification_report(Y_train,Y_train_pred))
print("Confusion matrix on Trained data - \n",confusion_matrix(Y_train,Y_train_pred))
print("Accuracy on Test data - ",accuracy_score(Y_test,Y_test_pred))
print("Classification report on Test data - \n",classification_report(Y_test,Y_test_pred))
print("Confusion matrix on Test data - \n",confusion_matrix(Y_test,Y_test_pred))

Using SVM :-
Accuracy on Trained data -  0.5012170834255366
Classification report on Trained data - 
               precision    recall  f1-score   support

           0       0.51      0.53      0.52      1374
           1       0.50      0.82      0.62      1865
           2       0.57      0.01      0.03       278
           3       0.00      0.00      0.00        14
           4       0.85      0.02      0.04       538
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00       207
           7       0.00      0.00      0.00       225
           8       0.00      0.00      0.00        11

    accuracy                           0.50      4519
   macro avg       0.27      0.15      0.13      4519
weighted avg       0.50      0.50      0.42      4519

Confusion matrix on Trained data - 
 [[ 725  648    1    0    0    0    0    0    0]
 [ 340 1525    0    0    0    0    0    0    0]
 [  66  208    4    0    0    0    0    0    0]
 [   4   10 

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metr

In [14]:
print("Using MLP :-")
model_MLP = MLP(X_train,Y_train)
Y_train_pred = model_MLP.predict(X_train)
Y_test_pred = model_MLP.predict(X_test)
print("Accuracy on Trained data - ",accuracy_score(Y_train,Y_train_pred))
print("Classification report on Trained data - \n",classification_report(Y_train,Y_train_pred))
print("Confusion matrix on Trained data - \n",confusion_matrix(Y_train,Y_train_pred))
print("Accuracy on Test data - ",accuracy_score(Y_test,Y_test_pred))
print("Classification report on Test data - \n",classification_report(Y_test,Y_test_pred))
print("Confusion matrix on Test data - \n",confusion_matrix(Y_test,Y_test_pred))

Using MLP :-
Accuracy on Trained data -  0.5543261783580438
Classification report on Trained data - 
               precision    recall  f1-score   support

           0       0.53      0.62      0.57      1374
           1       0.56      0.80      0.66      1865
           2       0.55      0.12      0.19       278
           3       0.50      0.21      0.30        14
           4       0.58      0.16      0.25       538
           5       1.00      0.14      0.25         7
           6       0.00      0.00      0.00       207
           7       0.76      0.21      0.33       225
           8       1.00      0.09      0.17        11

    accuracy                           0.55      4519
   macro avg       0.61      0.26      0.30      4519
weighted avg       0.54      0.55      0.51      4519

Confusion matrix on Trained data - 
 [[ 852  497    6    0   16    0    0    3    0]
 [ 360 1484    4    0   15    0    0    2    0]
 [  81  153   32    0    8    0    0    4    0]
 [   3    8 

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metr